In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
import os

In [5]:
schema_files = os.listdir('../schema')

In [6]:
all_schemas = {}

In [9]:
for file in schema_files:
    opened_file = open('../schema/' + file, 'r')
    all_schemas[file] = opened_file.read()

In [10]:
system_prompt = """You are a data engineer looking to create a slowly-changing dimension table query"""

In [12]:
user_prompt = f"""Using cumulative table input schema {all_schemas['product.sql']}
                    and expected output schema {all_schemas['product_scd_tbl.sql']} 
                    generate a query to do a slowly-changing dimension 
                    transformation tracking changes on the dimensions is_active and scoring_class, 
                    use markdown and SQL for the transformation
            """

In [13]:
print(system_prompt)
print(user_prompt)

You are a data engineer looking to create a slowly-changing dimension table query
Using cumulative table input schema CREATE TABLE product (
  Product_ID INT PRIMARY KEY,
  Category_Name VARCHAR(50),
  Sub_Category_Name VARCHAR(50),
  Brand VARCHAR(50),
  Feature_Desc VARCHAR(100)
)

                    and expected output schema create temp table #product_temp
select a.Product_ID as Product_ID_New,
case when a.Category_Name <> b.Category_Name then ‘-Category_Name’ else ‘’ end ||
case when a.Sub_Category_Name <> b.Sub_Category_Name then ‘-Sub_Category_Name’ else ‘’ end ||
case when a.Brand <> b.Brand then ‘-Brand’ else ‘’ end ||
case when a.Feature_Desc <> b.Feature_Desc then ‘-Feature_Desc’ else ‘’ end as CHANGED_COLUMN_NEW
from Dim_Product a join Stg_Product b
on a.Product_ID=b.Product_ID and a.current_flag=’Y’
where
a.Category_Name <> b.Category_Name or
a.Sub_Category_Name <> b.Sub_Category_Name or
a.Brand <> b.Brand or
a.Feature_Desc <> b.Feature_Desc;
 
                    generat

In [14]:
response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0
)
print(response.choices[0].message.content)
answer = response.choices[0].message.content

Here is a SQL query to create a slowly-changing dimension table that tracks changes on the dimensions `is_active` and `scoring_class`. 

First, let's assume that we have a staging table `Stg_Product` with the same schema as `product` table but with additional columns `is_active` and `scoring_class`.

```sql
CREATE TABLE Stg_Product (
  Product_ID INT PRIMARY KEY,
  Category_Name VARCHAR(50),
  Sub_Category_Name VARCHAR(50),
  Brand VARCHAR(50),
  Feature_Desc VARCHAR(100),
  is_active BOOLEAN,
  scoring_class VARCHAR(50)
)
```

Now, we can create a slowly-changing dimension table `Dim_Product` with additional columns `is_active`, `scoring_class`, `current_flag` and `changed_column`.

```sql
CREATE TABLE Dim_Product (
  Product_ID INT,
  Category_Name VARCHAR(50),
  Sub_Category_Name VARCHAR(50),
  Brand VARCHAR(50),
  Feature_Desc VARCHAR(100),
  is_active BOOLEAN,
  scoring_class VARCHAR(50),
  current_flag CHAR(1),
  changed_column VARCHAR(200)
)
```

The `current_flag` column is use

In [15]:
if not os.path.exists('output'):
    os.mkdir('output')

```sql
SELECT * FROM table
```

In [16]:
output = filter(lambda x: x.startswith('sql'), answer.split('```'))
# Open the file with write permissions
with open('output/product_scd_generation.sql', 'w') as file:
    # Write some data to the file
    file.write('\n'.join(output))